In [1]:
import pandas as pd
import numpy as np

Reading CSV File

In [2]:
df = pd.read_csv("./Final_IC.csv")
df.head(5)

,article_id,article_desc
0,Article 1 of Indian Constitution,"Name and territory of the Union India, that is..."
1,Article 2 of Indian Constitution,Admission or establishment of new States: Parl...
2,Article 2A of Indian Constitution,Sikkim to be associated with the Union Rep by ...
3,Article 3 of Indian Constitution,Formation of new States and alteration of area...
4,Article 4 of Indian Constitution,Laws made under Articles 2 and 3 to provide fo...


In [3]:
df['ques'] = 'What is ' + df['article_id'].astype(str)

Preprocessing of the text

In [4]:
df['ans'] = df['article_id'].astype(str) + ' states that ' + df['article_desc']
df.drop('article_id',inplace=True,axis=1)
df.drop('article_desc',inplace=True,axis = 1)

In [5]:
df.head(5)

,ques,ans
0,What is Article 1 of Indian Constitution,Article 1 of Indian Constitution states that N...
1,What is Article 2 of Indian Constitution,Article 2 of Indian Constitution states that A...
2,What is Article 2A of Indian Constitution,Article 2A of Indian Constitution states that ...
3,What is Article 3 of Indian Constitution,Article 3 of Indian Constitution states that F...
4,What is Article 4 of Indian Constitution,Article 4 of Indian Constitution states that L...


In [6]:
import tensorflow as tf
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences
Sequential = tf.keras.models.Sequential
Embedding = tf.keras.layers.Embedding
SimpleRNN = tf.keras.layers.SimpleRNN
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
Dropout = tf.keras.layers.Dropout

In [7]:
questions = df['ques'].tolist()
answers = df['ans'].tolist()

In [8]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(answers)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(answers)[0]

# Prepare input and target sequences
input_sequences = []
output_sequences = []

sequence_length = 100
for i in range(len(sequences) - sequence_length):
    input_sequences.append(sequences[i:i + sequence_length])
    output_sequences.append(sequences[i + sequence_length])

input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

vocab_size = len(tokenizer.word_index) + 1

In [9]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=sequence_length),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(vocab_size, activation="softmax"),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           1408      
                                                                 
 lstm (LSTM)                 (None, 100, 128)          82432     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 44)                5676      
                                                                 
Total params: 221100 (863.67 KB)
Trainable params: 221100 (863.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
epochs = 50
batch_size = 32
model.fit(input_sequences, output_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/50
9/9 [==============================] - 3s 320ms/step - loss: 2.7152 - accuracy: 0.1583
Epoch 2/50
9/9 [==============================] - 5s 553ms/step - loss: 2.7088 - accuracy: 0.1475
Epoch 3/50
9/9 [==============================] - 4s 467ms/step - loss: 2.7138 - accuracy: 0.1439
Epoch 4/50
9/9 [==============================] - 5s 600ms/step - loss: 2.7066 - accuracy: 0.1547
Epoch 5/50
9/9 [==============================] - 6s 644ms/step - loss: 2.7031 - accuracy: 0.1583
Epoch 6/50
9/9 [==============================] - 6s 657ms/step - loss: 2.7131 - accuracy: 0.1583
Epoch 7/50
9/9 [==============================] - 6s 638ms/step - loss: 2.7119 - accuracy: 0.1439
Epoch 8/50
9/9 [==============================] - 6s 639ms/step - loss: 2.7039 - accuracy: 0.1439
Epoch 9/50
9/9 [==============================] - 6s 647ms/step - loss: 2.7192 - accuracy: 0.1583
Epoch 10/50
9/9 [==============================] - 6s 636ms/step - loss: 2.7025 - accuracy: 0.1547
Epoch 11/50
9/9 [==

In [17]:
def generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate):
    generated_text = seed_text

    for _ in range(num_chars_to_generate):
        token_list = tokenizer.texts_to_sequences([generated_text])
        token_list = pad_sequences(token_list, maxlen=sequence_length, padding="pre")
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_token = np.argmax(predicted_probs, axis=-1)[0]  # Get the index of the predicted token

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_token:
                output_word = word
                break

        generated_text += output_word

    return generated_text


In [18]:
seed_text = "Article 2 of Indian Constitution"

generated_text = generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate=150)
print(generated_text)

Article 2 of Indian Constitution territories the territories the territories the territories the territories the territories the territories the territories the territories the terri
